## Dependências

Dependências que serão usadas por todas as partes

In [17]:
!pip install psycopg2-binary ipython-sql pandas sqlalchemy
import pandas as pd
import psycopg2
import os
import sys
import subprocess


# Parte I

## Tarefa 1 – Identificação do Sistema

- Tipo de processador:

- Quantidade de memória RAM:

- Tamanho do disco:

- Informações sobre as caches existentes:

- Distribuição usada:

- Versão do sistema:

- Versão do Kernel:

Distribuição/Versão do SO	    lsb_release -a ou cat /etc/os-release (mais completo as informaçoes)

Versão do Kernel	            uname -r

Tipo de Processador	            lscpu ou cat /proc/cpuinfo (muita informaçao)

Caches do Processador	        lscpu ou `getconf -a

Memória RAM	                    free -h ou cat /proc/meminfo

Tamanho do Disco	            lsblk ou df -h

## Tarefa 2 - Verificação de parâmetros de armazenamento

a) Parâmetros do Disco (no laboratório):

- Número de superfícies 

- Cilindros 

- Setores por trilha 

- Velocidade de rotação 

- Latência rotacional 

- Tempos de seek (médio, máximo e mínimo) 

- Tempo para a próxima trilha 

- Taxa de transferência 

b) Parâmetros de S.O. para o Disco:

- Verificar os parâmetros de S.O. que serão utilizados para o disco, usando o comando Linux stat.

c) Blocos:

- Verificar o tamanho de bloco utilizado.

- Mostrar como alterar o tamanho dos blocos.


O que entregar: Os resultados de cada verificação devem ser preenchidos no Jupyter Notebook.

# Tarefa 3 – Geração de um BD para testes



### Clonando os repositórios necessários

In [12]:
# clona o repositório se não existir
if not os.path.exists('tpch-pgsql'):
    print("Clonando repositório tpch-pgsql...")
    !git clone https://github.com/foliveirafilho/tpch-pgsql.git
    
# Entra na pasta do projeto para os comandos seguintes funcionarem
os.chdir('tpch-pgsql')
!pip install -r requirements.txt
print(f"Diretório de trabalho atual: {os.getcwd()}")
if not os.path.exists('tpch-dbgen'):
    print("Clonando a ferramenta geradora de dados (tpch-dbgen)...")
    !git clone https://github.com/electrum/tpch-dbgen.git
else:
    print("Ferramenta tpch-dbgen já foi clonada.")

Clonando repositório tpch-pgsql...
Cloning into 'tpch-pgsql'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 160 (from 1)
Receiving objects: 100% (164/164), 774.38 KiB | 819.00 KiB/s, done.
Resolving deltas: 100% (69/69), done.
Diretório de trabalho atual: /home/lucas/faculdade/bd/Banco_de_Dados_tp3/tpch-pgsql/tpch-pgsql
Clonando a ferramenta geradora de dados (tpch-dbgen)...
Cloning into 'tpch-dbgen'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 149 (delta 10), reused 8 (delta 8), pack-reused 120 (from 1)
Receiving objects: 100% (149/149), 215.40 KiB | 407.00 KiB/s, done.
Resolving deltas: 100% (35/35), done.


### Criando o Banco de Dados

In [18]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

db_config = {
    'user': 'postgres',                
    'password': 'postgres', 
    'host': 'localhost',
    'port': '5432'
}
db_name = "trabalho_bd3"

# Conexão ao postgres para criar o novo banco
print(f"Conectando ao PostgreSQL para criar o banco '{db_name}'...")
conn = psycopg2.connect(dbname='postgres', **db_config)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

cur.execute(f"DROP DATABASE IF EXISTS {db_name};") # para que não haja o mesmo DB se for executado mais de uma vez
print(f"Banco antigo '{db_name}' removido (se existia).")

cur.execute(f"CREATE DATABASE {db_name};")
print(f"Banco novo '{db_name}' criado com sucesso!")

cur.close()
conn.close()

# Criação da Engine do SQLAlchemy 
connection_str = (
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_name}"
)
engine = create_engine(connection_str)
print("Engine SQLAlchemy configurada com sucesso.")

Conectando ao PostgreSQL para criar o banco 'trabalho_bd3'...
Banco antigo 'trabalho_bd3' removido (se existia).
Banco novo 'trabalho_bd3' criado com sucesso!
Engine SQLAlchemy configurada com sucesso.


### Gerando os códigos para a tarefa

In [19]:
SCALE_FACTOR = "0.1"
print(f"Diretório atual: {os.getcwd()}")
if not os.path.exists("tpch_pgsql.py"):
    print("ALERTA: tpch_pgsql.py não encontrado neste diretório. Verifique o caminho!")

Diretório atual: /home/lucas/faculdade/bd/Banco_de_Dados_tp3/tpch-pgsql/tpch-pgsql


In [20]:
cmd_prepare = [
    "python3", "tpch_pgsql.py",
    "--host", db_config['host'],
    "--user", db_config['user'],
    "--password", db_config['password'],
    "--dbname", db_name,
    "prepare",         
    "--scale", SCALE_FACTOR
]

# Usando Popen para ver o output em tempo real no notebook
with subprocess.Popen(cmd_prepare, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
    for line in proc.stdout:
        print(line, end='')

if proc.returncode != 0:
    print("\n Erro na geração dos dados. Verifique o log acima.")
else:
    print("\n Dados gerados com sucesso!")

make: Nada a ser feito para 'all'.
built dbgen from source
TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating data for suppliers table
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

### Carregando os dados necessários para a tarefa

In [21]:
cmd_load = [
    "python3", "tpch_pgsql.py",
    "--host", db_config['host'],
    "--user", db_config['user'],
    "--password", db_config['password'],
    "--dbname", db_name,
    "load"
]

print(f"Iniciando Fase LOAD no banco '{db_name}'... Isso pode demorar alguns minutos.")
print("-" * 60)

with subprocess.Popen(cmd_load, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
    for line in proc.stdout:
        print(line, end='')

print("\n" + "-" * 60)
print("Carga concluída! O banco está pronto para as consultas.")

Iniciando Fase LOAD no banco 'trabalho_bd3'... Isso pode demorar alguns minutos.
------------------------------------------------------------
dropped existing tables
cleaned database trabalho_bd3
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.014905
load_data: 0:00:00.979366
index_tables: 0:00:00.770025
======================= End Results ========================

------------------------------------------------------------
Carga concluída! O banco está pronto para as consultas.


### Execução de Querrys

In [22]:
import pandas as pd
from sqlalchemy import text
import time

def analisar_query(nome, sql, engine_conexao):
    print(f"=== {nome} ===")
    
    print("--- Plano de Execução ---")
    with engine_conexao.connect() as conn:
        explain = conn.execute(text(f"EXPLAIN ANALYZE {sql}"))
        for row in explain.fetchall()[:20]:
            print(row[0])
            
    # Execução Real e Tempo
    print("\n--- Resultado (Top 10) ---")
    start = time.time()
    df = pd.read_sql_query(text(sql), engine_conexao)
    tempo = time.time() - start
    
    display(df.head(10))
    print(f"Tempo Total: {tempo:.4f} segundos\n")
    print("="*50)



# Q1: Pricing Summary Report
# Foca em agregações e scans na tabela lineitem
sql_q1 = """
SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) as sum_qty,
    sum(l_extendedprice) as sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
    avg(l_quantity) as avg_qty,
    avg(l_extendedprice) as avg_price,
    avg(l_discount) as avg_disc,
    count(*) as count_order
FROM
    lineitem
WHERE
    l_shipdate <= date '1998-12-01' - interval '90 days'
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
"""
analisar_query("Q1 - Pricing Summary", sql_q1, engine)


# Q3: Shipping Priority 
# Teste de junção (Join) entre Customer, Orders e Lineitem
sql_q3 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_mktsegment = 'BUILDING'
    AND c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate < date '1995-03-15'
    AND l_shipdate > date '1995-03-15'
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue desc,
    o_orderdate
LIMIT 10;
"""
analisar_query("Q3 - Shipping Priority", sql_q3, engine)


# Q5: Local Supplier Volume
# Query complexa envolvendo junção de 6 tabelas (Customer, Orders, Lineitem, Supplier, Nation, Region)
sql_q5 = """
SELECT
    n_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue
FROM
    customer,
    orders,
    lineitem,
    supplier,
    nation,
    region
WHERE
    c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= date '1994-01-01'
    AND o_orderdate < date '1994-01-01' + interval '1 year'
GROUP BY
    n_name
ORDER BY
    revenue desc;
"""
analisar_query("Q5 - Local Supplier Volume", sql_q5, engine)


# Q6: Forecasting Revenue Change
# Scan simples na lineitem com múltiplos filtros (quantidade, desconto e data)
# Observar se o otimizador usa índices ou scan sequencial
sql_q6 = """
SELECT
    sum(l_extendedprice * l_discount) as revenue
FROM
    lineitem
WHERE
    l_shipdate >= date '1994-01-01'
    AND l_shipdate < date '1994-01-01' + interval '1 year'
    AND l_discount between 0.06 - 0.01 AND 0.06 + 0.01
    AND l_quantity < 24;
"""
analisar_query("Q6 - Forecasting Revenue", sql_q6, engine)


# Q7: Volume Shipping
# Analisa o volume de mercadorias entre duas nações específicas 
# Envolve subquery ou lógica complexa de OR/AND nas nações
sql_q7 = """
SELECT
    supp_nation,
    cust_nation,
    l_year,
    sum(volume) as revenue
FROM
    (
        SELECT
            n1.n_name as supp_nation,
            n2.n_name as cust_nation,
            extract(year from l_shipdate) as l_year,
            l_extendedprice * (1 - l_discount) as volume
        FROM
            supplier,
            lineitem,
            orders,
            customer,
            nation n1,
            nation n2
        WHERE
            s_suppkey = l_suppkey
            AND o_orderkey = l_orderkey
            AND c_custkey = o_custkey
            AND s_nationkey = n1.n_nationkey
            AND c_nationkey = n2.n_nationkey
            AND (
                (n1.n_name = 'FRANCE' AND n2.n_name = 'GERMANY')
                OR (n1.n_name = 'GERMANY' AND n2.n_name = 'FRANCE')
            )
            AND l_shipdate between date '1995-01-01' AND date '1996-12-31'
    ) as shipping
GROUP BY
    supp_nation,
    cust_nation,
    l_year
ORDER BY
    supp_nation,
    cust_nation,
    l_year;
"""
analisar_query("Q7 - Volume Shipping", sql_q7, engine)


# Q9: Product Type Profit Measure
# Query pesada. Envolve junção de Part, Supplier, Lineitem, Partsupp, Orders, Nation
# Filtra por partes que contenham 'green' no nome
sql_q9 = """
SELECT
    nation,
    o_year,
    sum(amount) as sum_profit
FROM
    (
        SELECT
            n_name as nation,
            extract(year from o_orderdate) as o_year,
            l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
        FROM
            part,
            supplier,
            lineitem,
            partsupp,
            orders,
            nation
        WHERE
            s_suppkey = l_suppkey
            AND ps_suppkey = l_suppkey
            AND ps_partkey = l_partkey
            AND p_partkey = l_partkey
            AND o_orderkey = l_orderkey
            AND s_nationkey = n_nationkey
            AND p_name like '%green%'
    ) as profit
GROUP BY
    nation,
    o_year
ORDER BY
    nation,
    o_year desc;
"""
analisar_query("Q9 - Product Type Profit", sql_q9, engine)


# Q10: Returned Item Reporting
# Identifica clientes que retornaram peças
# Top 20 clientes por receita perdida
sql_q10 = """
SELECT
    c_custkey,
    c_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    c_acctbal,
    n_name,
    c_address,
    c_phone,
    c_comment
FROM
    customer,
    orders,
    lineitem,
    nation
WHERE
    c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate >= date '1993-10-01'
    AND o_orderdate < date '1993-10-01' + interval '3 months'
    AND l_returnflag = 'R'
    AND c_nationkey = n_nationkey
GROUP BY
    c_custkey,
    c_name,
    c_acctbal,
    c_phone,
    n_name,
    c_address,
    c_comment
ORDER BY
    revenue desc
LIMIT 20;
"""
analisar_query("Q10 - Returned Item Reporting", sql_q10, engine)


# Q12: Shipping Modes and Order Priority
# Analisa se modos de envio baratos (MAIL, SHIP) afetam a prioridade de entrega
sql_q12 = """
SELECT
    l_shipmode,
    sum(case
        when o_orderpriority = '1-URGENT'
            OR o_orderpriority = '2-HIGH'
            then 1
        else 0
    end) as high_line_count,
    sum(case
        when o_orderpriority <> '1-URGENT'
            AND o_orderpriority <> '2-HIGH'
            then 1
        else 0
    end) as low_line_count
FROM
    orders,
    lineitem
WHERE
    o_orderkey = l_orderkey
    AND l_shipmode in ('MAIL', 'SHIP')
    AND l_commitdate < l_receiptdate
    AND l_shipdate < l_commitdate
    AND l_receiptdate >= date '1994-01-01'
    AND l_receiptdate < date '1994-01-01' + interval '1 year'
GROUP BY
    l_shipmode
ORDER BY
    l_shipmode;
"""
analisar_query("Q12 - Shipping Modes", sql_q12, engine)

=== Q1 - Pricing Summary ===
--- Plano de Execução ---
Finalize GroupAggregate  (cost=24055.44..24057.40 rows=6 width=236) (actual time=120.518..121.945 rows=4 loops=1)
  Group Key: l_returnflag, l_linestatus
  ->  Gather Merge  (cost=24055.44..24056.84 rows=12 width=236) (actual time=120.505..121.915 rows=12 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=23055.42..23055.43 rows=6 width=236) (actual time=118.760..118.761 rows=4 loops=3)
              Sort Key: l_returnflag, l_linestatus
              Sort Method: quicksort  Memory: 26kB
              Worker 0:  Sort Method: quicksort  Memory: 26kB
              Worker 1:  Sort Method: quicksort  Memory: 26kB
              ->  Partial HashAggregate  (cost=23055.20..23055.34 rows=6 width=236) (actual time=118.733..118.737 rows=4 loops=3)
                    Group Key: l_returnflag, l_linestatus
                    Batches: 1  Memory Usage: 24kB
                    Worker 0:  Batches: 1  Memory Usa

,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,3774200.0,5.320754e+09,5.054096e+09,5.256751e+09,25.537587,36002.123829,0.050145,147790
1,N,F,95257.0,1.337378e+08,1.271324e+08,1.322863e+08,25.300664,35521.326916,0.049394,3765
2,N,O,7459297.0,1.051227e+10,9.986238e+09,1.038558e+10,25.545538,36000.924688,0.050096,292000
3,R,F,3785523.0,5.337951e+09,5.071819e+09,5.274406e+09,25.525944,35994.029214,0.049989,148301


Tempo Total: 0.1215 segundos

=== Q3 - Shipping Priority ===
--- Plano de Execução ---
Limit  (cost=21201.80..21201.82 rows=10 width=44) (actual time=39.054..40.802 rows=10 loops=1)
  ->  Sort  (cost=21201.80..21283.26 rows=32585 width=44) (actual time=39.053..40.801 rows=10 loops=1)
        Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC, orders.o_orderdate
        Sort Method: top-N heapsort  Memory: 26kB
        ->  HashAggregate  (cost=20090.33..20497.65 rows=32585 width=44) (actual time=38.604..40.612 rows=1216 loops=1)
              Group Key: lineitem.l_orderkey, orders.o_orderdate, orders.o_shippriority
              Batches: 1  Memory Usage: 1297kB
              ->  Gather  (cost=1587.81..19601.56 rows=32585 width=24) (actual time=1.451..39.105 rows=3321 loops=1)
                    Workers Planned: 1
                    Workers Launched: 1
                    ->  Nested Loop  (cost=587.81..15343.06 rows=19168 width=24) (actual time=1.41

,l_orderkey,revenue,o_orderdate,o_shippriority
0,223140,355369.0698,1995-03-14,0
1,584291,354494.7318,1995-02-21,0
2,405063,353125.4577,1995-03-03,0
3,573861,351238.2770,1995-03-09,0
4,554757,349181.7426,1995-03-14,0
5,506021,321075.5810,1995-03-10,0
6,121604,318576.4154,1995-03-07,0
7,108514,314967.0754,1995-02-20,0
8,462502,312604.5420,1995-03-08,0
9,178727,309728.9306,1995-02-25,0


Tempo Total: 0.0241 segundos

=== Q5 - Local Supplier Volume ===
--- Plano de Execução ---
Sort  (cost=12888.50..12888.56 rows=25 width=136) (actual time=15.789..18.249 rows=5 loops=1)
  Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC
  Sort Method: quicksort  Memory: 25kB
  ->  Finalize GroupAggregate  (cost=12878.87..12887.92 rows=25 width=136) (actual time=15.711..18.239 rows=5 loops=1)
        Group Key: nation.n_name
        ->  Gather Merge  (cost=12878.87..12887.42 rows=25 width=136) (actual time=15.676..18.229 rows=10 loops=1)
              Workers Planned: 1
              Workers Launched: 1
              ->  Partial GroupAggregate  (cost=11878.86..11884.59 rows=25 width=136) (actual time=14.148..14.226 rows=5 loops=2)
                    Group Key: nation.n_name
                    ->  Sort  (cost=11878.86..11879.94 rows=434 width=116) (actual time=14.114..14.128 rows=432 loops=2)
                          Sort Key: nation.n_name
      

...

Tempo Total: 0.0426 segundos

=== Q10 - Returned Item Reporting ===
--- Plano de Execução ---
Limit  (cost=21294.61..21294.66 rows=20 width=280) (actual time=35.085..37.749 rows=20 loops=1)
  ->  Sort  (cost=21294.61..21308.50 rows=5554 width=280) (actual time=35.084..37.747 rows=20 loops=1)
        Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC
        Sort Method: top-N heapsort  Memory: 33kB
        ->  Finalize GroupAggregate  (cost=20433.30..21146.82 rows=5554 width=280) (actual time=28.185..36.711 rows=3767 loops=1)
              Group Key: customer.c_custkey, nation.n_name
              ->  Gather Merge  (cost=20433.30..21031.12 rows=4628 width=280) (actual time=28.179..33.930 rows=4462 loops=1)
                    Workers Planned: 2
                    Workers Launched: 2
                    ->  Partial GroupAggregate  (cost=19433.27..19496.91 rows=2314 width=280) (actual time=26.101..28.153 rows=1487 loops=3)
                          G

,c_custkey,c_name,revenue,c_acctbal,n_name,c_address,c_phone,c_comment
0,8242,Customer#000008242,622786.7297,6322.09,ETHIOPIA,"P2n4nJhy,UqSo2s43YfSvYJDZ6lk",15-792-676-1184,slyly regular packages haggle carefully ironic...
1,7714,Customer#000007714,557400.3053,9799.98,IRAN,"SnnIGB,SkmnWpX3",20-922-418-6024,arhorses according to the blithely express re
2,11032,Customer#000011032,512500.9641,8496.93,UNITED KINGDOM,"WIKHC7K3Cn7156iNOyfVG3cZ7YqkgsR,Ly",33-102-772-3533,posits-- furiously ironic accounts are again
3,2455,Customer#000002455,481592.4053,2070.99,GERMANY,RVn1ZSRtLqPlJLIZxvpmsbgC02,17-946-225-9977,al asymptotes. finally ironic accounts cajole ...
4,12106,Customer#000012106,479414.2133,5342.11,UNITED STATES,wth3twOmu6vy,34-905-346-4472,ly after the blithely regular foxes. accounts ...
5,8530,Customer#000008530,457855.9467,9734.95,MOROCCO,GMQyte94oDM7eD7exnkj 4hH9yq3,25-736-932-5850,slyly asymptotes. quickly final deposits in
6,13984,Customer#000013984,446316.5104,3482.28,IRAN,qZXwuapCHvxbX,20-981-264-2952,y unusual courts could wake furiously
7,1966,Customer#000001966,444059.0382,1937.72,ALGERIA,jPv1 UHra5JLALR5Isci5u0636RoAu7t vH,10-973-269-8886,the blithely even accounts. final deposits caj...
8,11026,Customer#000011026,417913.4142,7738.76,ALGERIA,XorIktoJOAEJkpNNMx,10-184-163-4632,ly even dolphins eat along the blithely even i...
9,8501,Customer#000008501,412797.5100,6906.70,ARGENTINA,776af4rOa mZ66hczs,11-317-552-5840,y final deposits after the fluffily even accou...


Tempo Total: 0.0452 segundos

=== Q12 - Shipping Modes ===
--- Plano de Execução ---
Finalize GroupAggregate  (cost=22007.48..22043.60 rows=7 width=27) (actual time=39.679..42.309 rows=2 loops=1)
  Group Key: lineitem.l_shipmode
  ->  Gather Merge  (cost=22007.48..22043.47 rows=7 width=27) (actual time=39.564..42.304 rows=4 loops=1)
        Workers Planned: 1
        Workers Launched: 1
        ->  Partial GroupAggregate  (cost=21007.47..21042.68 rows=7 width=27) (actual time=38.121..38.223 rows=2 loops=2)
              Group Key: lineitem.l_shipmode
              ->  Sort  (cost=21007.47..21011.86 rows=1757 width=27) (actual time=38.012..38.048 rows=1578 loops=2)
                    Sort Key: lineitem.l_shipmode
                    Sort Method: quicksort  Memory: 140kB
                    Worker 0:  Sort Method: quicksort  Memory: 130kB
                    ->  Parallel Hash Join  (cost=16933.92..20912.77 rows=1757 width=27) (actual time=31.466..37.789 rows=1578 loops=2)
              

,l_shipmode,high_line_count,low_line_count
0,MAIL,647,945
1,SHIP,620,943


Tempo Total: 0.0457 segundos



### Comentários breves os tipos de acesso observados:

Varredura Sequencial (Sequential Scan / Parallel Seq Scan):

- Onde ocorre: Predominante nas Queries Q1 e Q6.

- Por quê: Essas consultas realizam agregações (SUM, AVG) ou filtros (WHERE) em grandes intervalos de datas na tabela massiva lineitem. O otimizador geralmente prefere ler a tabela inteira sequencialmente (usando múltiplas threads/workers) do que fazer milhões de acessos aleatórios em um índice.

Junção por Hash (Hash Join):

- Onde ocorre: Fundamental nas Queries Q3, Q5, Q9, Q10 e Q12.

- Por quê: Ao unir tabelas grandes não ordenadas (ex: customer, orders e lineitem), o PostgreSQL cria uma tabela hash em memória para a menor tabela e varre a maior para encontrar correspondências. É o método mais rápido para grandes volumes de dados não indexados pela chave de junção.

Junção por Laço Aninhado (Nested Loop):

- Onde pode ocorrer: Possível nas Queries Q5 e Q7 ao acessar tabelas muito pequenas (tabelas dimensionais).

- Por quê: Ao unir tabelas como nation ou region (que têm pouquíssimas linhas) com o restante do conjunto, o otimizador pode escolher o Nested Loop por ser mais eficiente para conjuntos de dados minúsculos.

Ordenação (Sort / GroupAggregate):

- Onde ocorre: Em quase todas (Q1, Q3, Q5, Q7, Q9, Q10, Q12).

- Por quê: Necessário para resolver as cláusulas GROUP BY e ORDER BY. Após filtrar e juntar os dados, o banco precisa ordenar o resultado final, muitas vezes usando o método Quicksort em memória ou em disco se o volume for muito grande.

# Parte II

## Configurando a conexão, pode pular se o notebook já estiver conectado

In [ ]:
# pode pular essa parte se vc estiver executando isso junto com a Parte I
# está se conectando a um banco JÁ EXISTENTE!

from sqlalchemy import create_engine, text

db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'  
}

db_name = 'trabalho_bd3'  

engine = create_engine(
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_name}"
)

def run_query(query):
    with engine.connect() as conn:
        result = conn.execute(text(query))
        return result.fetchall()

print(f"Engine configurada para o banco: {db_name}")

Engine configurada para o banco: trabalho_bd3


## Tarefa 5 – Preparação da Tabela Aleatória

In [2]:
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS t;"))
    conn.execute(text("""
        CREATE TABLE t (
            k SERIAL PRIMARY KEY,
            v INTEGER
        );
    """))

with engine.begin() as conn:
    conn.execute(text("""
        INSERT INTO t(v)
        SELECT trunc(random() * 10)
        FROM generate_series(1, 100000);
    """))

primeiros = run_query("SELECT * FROM t ORDER BY k LIMIT 10;")
for linha in primeiros:
    print(linha)


(1, 3)
(2, 3)
(3, 2)
(4, 6)
(5, 4)
(6, 9)
(7, 4)
(8, 7)
(9, 2)
(10, 3)


## Tarefa 6 - Páginas criadas

In [3]:
#Atualizar as estatísticas da tabela (comando sem retorno de linhas):
with engine.begin() as conn:
    conn.execute(text("ANALYZE t;"))

#Consultar pg_class para ver o número estimado de páginas e tuplas:
pages_info = run_query("""
    SELECT relname, relpages, reltuples
    FROM pg_class
    WHERE relname = 't';
""")
print(pages_info)


[('t', 443, 100000.0)]


## Tarefa 7 - Blocos

In [4]:
with engine.begin() as conn:
    conn.execute(text("SELECT pg_stat_reset();"))
resultado = run_query("SELECT * FROM t WHERE v = 5;")

stats_io = run_query("""
    SELECT relname,
           heap_blks_read, heap_blks_hit,
           idx_blks_read, idx_blks_hit
    FROM pg_statio_user_tables
    WHERE relname = 't';
""")
stats_io

stats_access = run_query("""
    SELECT relname, seq_scan, seq_tup_read, idx_scan, idx_tup_fetch
    FROM pg_stat_user_tables
    WHERE relname = 't';
""")
stats_access


[('t', 0, 0, 0, 0)]

## Tarefa 8 - Índice

### Com a tabela inicial

In [12]:
import time

# Medindo o tempo de uma consulta antes de criar o índice
valor_procurado = 7  # escolha qualquer valor de 0 a 9
inicio = time.perf_counter()
run_query(f"SELECT * FROM t WHERE v = {valor_procurado};")
fim = time.perf_counter()
tempo_sem_indice = fim - inicio
print(f"Tempo da consulta sem índice: {tempo_sem_indice:.4f} segundos")

# Criando o índice e medindo o tempo de criação
inicio = time.perf_counter()
with engine.begin() as conn:
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_t_v ON t(v);") )
fim = time.perf_counter()
tempo_criacao_indice = fim - inicio
print(f"Tempo para criar o índice: {tempo_criacao_indice:.4f} segundos")

# Medindo o tempo da mesma consulta após criar o índice
inicio = time.perf_counter()
run_query(f"SELECT * FROM t WHERE v = {valor_procurado};")
fim = time.perf_counter()
tempo_com_indice = fim - inicio
print(f"Tempo da consulta com índice: {tempo_com_indice:.4f} segundos")

# Exibindo o plano de execução (EXPLAIN ANALYZE) para entender a diferença
plan = run_query(f"EXPLAIN ANALYZE SELECT * FROM t WHERE v = {valor_procurado};")
for linha in plan:
    print(linha[0])


Tempo da consulta sem índice: 0.0758 segundos
Tempo para criar o índice: 0.0004 segundos
Tempo da consulta com índice: 0.1080 segundos
Bitmap Heap Scan on t  (cost=1113.88..6786.38 rows=99800 width=8) (actual time=1.582..11.279 rows=100000 loops=1)
  Recheck Cond: (v = 7)
  Heap Blocks: exact=4425
  ->  Bitmap Index Scan on idx_t_v  (cost=0.00..1088.92 rows=99800 width=0) (actual time=1.197..1.198 rows=100000 loops=1)
        Index Cond: (v = 7)
Planning Time: 0.046 ms
Execution Time: 12.711 ms


### Com a tabela de 1.000.000 de elementos

In [13]:
# Recria a tabela 't' com 1 milhão de linhas

import time

# Remove o índice existente, se houver, e a tabela
with engine.begin() as conn:
    conn.execute(text("DROP INDEX IF EXISTS idx_t_v;"))
    conn.execute(text("DROP TABLE IF EXISTS t;"))

# Cria a tabela
with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE t (
            k SERIAL PRIMARY KEY,
            v INTEGER
        );
    """))

# Insere 1.000.000 de linhas
with engine.begin() as conn:
    conn.execute(text("""
        INSERT INTO t(v)
        SELECT trunc(random() * 10)
        FROM generate_series(1, 1000000);
    """))

# Mede o tempo da consulta sem índice
valor_procurado = 7
inicio = time.perf_counter()
run_query(f"SELECT * FROM t WHERE v = {valor_procurado};")
fim = time.perf_counter()
tempo_sem_indice_large = fim - inicio
print(f"Tempo da consulta sem índice (1M linhas): {tempo_sem_indice_large:.4f} segundos")

# Cria o índice e mede o tempo de criação
inicio = time.perf_counter()
with engine.begin() as conn:
    conn.execute(text("CREATE INDEX idx_t_v ON t(v);") )
fim = time.perf_counter()
tempo_criacao_indice_large = fim - inicio
print(f"Tempo para criar o índice (1M linhas): {tempo_criacao_indice_large:.4f} segundos")

# Mede o tempo da consulta com índice
inicio = time.perf_counter()
run_query(f"SELECT * FROM t WHERE v = {valor_procurado};")
fim = time.perf_counter()
tempo_com_indice_large = fim - inicio
print(f"Tempo da consulta com índice (1M linhas): {tempo_com_indice_large:.4f} segundos")


Tempo da consulta sem índice (1M linhas): 0.0778 segundos
Tempo para criar o índice (1M linhas): 0.2116 segundos
Tempo da consulta com índice (1M linhas): 0.0687 segundos


## Tarefa 9 - fill factor

In [14]:
import time

# Definição dos parâmetros pedidos na Tarefa 9
fillfactors = [60, 80, 90, 100]

print(f"{'FF (%)':<8} | {'Criação (s)':<12} | {'Consulta (ms)':<14} | {'Tamanho (bytes)'}")
print("-" * 60)

with engine.connect() as conn:
    for ff in fillfactors:
        # Remove o índice anterior para não interferir
        conn.execute(text("DROP INDEX IF EXISTS idx_t_v"))
        conn.commit() # Necessário para efetivar o DROP antes de criar o próximo
        
        # Criação
        start_time = time.time()
        conn.execute(text(f"CREATE INDEX idx_t_v ON t(v) WITH (fillfactor = {ff})"))
        conn.commit()
        creation_time = time.time() - start_time
        
        # Consulta: Usa EXPLAIN ANALYZE para pegar o tempo real de execução
        result = conn.execute(text("EXPLAIN (ANALYZE, FORMAT JSON) SELECT * FROM t WHERE v = 5"))
        plan = result.scalar()
        exec_time = plan[0]['Execution Time']
        
        # Tamanho: Pega o tamanho físico para justificar o relatório
        size_result = conn.execute(text("SELECT pg_relation_size('idx_t_v')"))
        size_bytes = size_result.scalar()
        
        # Imprime a linha da tabela
        print(f"{ff:<8} | {creation_time:<12.4f} | {exec_time:<14.4f} | {size_bytes}")

FF (%)   | Criação (s)  | Consulta (ms)  | Tamanho (bytes)
------------------------------------------------------------
60       | 0.2278       | 16.3860        | 10412032
80       | 0.2095       | 12.2780        | 7815168
90       | 0.2115       | 19.6310        | 6963200
100      | 0.2427       | 12.0500        | 6963200


### Relatório da Tarefa 9: Impacto do Fillfactor

**Resultados Observados:**
Ao testar os fillfactors 60, 80, 90 e 100, observou-se que o índice com **fillfactor 100** ocupou o menor espaço em disco (menor número de bytes), pois as páginas foram preenchidas completamente.

**Conclusão:**
Embora tempos de consulta (SELECT) muito similares tenham sido observados (diferenças na casa dos milissegundos), o **fillfactor 100** é ideal para tabelas estáticas (apenas leitura), maximizando o uso do espaço. Já os fillfactors menores (**60 a 90**) geraram índices fisicamente maiores, sendo mais indicados para tabelas com muitas atualizações, pois o espaço livre deixado evita a fragmentação e o custo de realocação de páginas durante INSERTs futuros.

# Parte III

## Conectando ao DB, pode pular se já tiver feito isso

In [23]:
# pode pular essa parte se vc estiver executando isso junto com a Parte I ou com a parte II
# está se conectando a um banco JÁ EXISTENTE!

from sqlalchemy import create_engine, text

db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'  
}

db_name = 'trabalho_bd3'  

engine = create_engine(
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_name}"
)

def run_query(query):
    with engine.connect() as conn:
        result = conn.execute(text(query))
        return result.fetchall()

print(f"Engine configurada para o banco: {db_name}")

Engine configurada para o banco: trabalho_bd3


## Tarefa 11 - Preparação e Verificação do Ambient

### 11.a)

In [24]:
try:
    # Lê o arquivo de forma segura
    with open('movie.sql', 'r') as f:
        sql_script = f.read()
    
    with engine.begin() as conn:
        conn.execute(text(sql_script))
    print("Sucesso: Tabela 'movie' recriada e dados inseridos!")

except psycopg2.errors.UndefinedTable:
    # Esse erro acontece se o script tiver "DROP TABLE movie" mas a tabela não existir
    print("Erro: O script tentou apagar uma tabela que não existe.")
    print("Dica: Altere a primeira linha do movie.sql para 'DROP TABLE IF EXISTS movie;'")
except Exception as e:
    print(f"Ocorreu um erro ao rodar o script: {e}")

Sucesso: Tabela 'movie' recriada e dados inseridos!


### 11.b)

In [ ]:
with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS pageinspect;"))
    print("Executando VACUUM ANALYZE na tabela 'movie'...")
    conn.execute(text("VACUUM ANALYZE movie;"))
    print("Estatísticas atualizadas com sucesso!")

sql_querry = """
SELECT
    t.relname AS tabela,
    i.relname AS indice,
    pg_size_pretty(pg_relation_size(i.oid)) AS tamanho,
    meta.level AS altura_arvore,     -- Nível da raiz (0 = folha, >0 = ramos)
    meta.root AS bloco_raiz,         -- Qual bloco é a raiz
    c.relpages AS total_blocos,      -- Total de páginas (blocos) do índice
    c.reltuples AS total_linhas      -- Estimativa de linhas indexadas
FROM pg_class t
JOIN pg_index x ON t.oid = x.indrelid
JOIN pg_class c ON c.oid = x.indexrelid
JOIN pg_class i ON i.oid = x.indexrelid
-- A função bt_metap lê a 'meta página' do índice B-Tree para pegar a altura
CROSS JOIN LATERAL bt_metap(i.relname) as meta
WHERE t.relname = 'movie';
"""

try: 
    df_resultado = pd.read_sql(sql_querry, engine)

    print("Relatório de Metadados do Índice")
    display(df_resultado)

    nome_arquivo = "relatorio_tarefa_11.txt"
    
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        
        f.write("BANCOS DE DADOS 1 - TP3 - TAREFA 11\n")
        f.write("="*50 + "\n\n")
        f.write("RELATÓRIO DE METADADOS DOS ÍNDICES:\n\n")
        
        # O método .to_string() converte a tabela do Pandas para texto puro
        # index=False remove a numeração das linhas (0, 1, 2...)
        tabela_texto = df_resultado.to_string(index=False, justify='left')
        
        f.write(tabela_texto)
        f.write("\n\n" + "="*50)
        
    print(f"\nSucesso! O arquivo '{nome_arquivo}' foi salvo na pasta do notebook.")

except Exception as e:
    print(f"Erro na consulta: {e}")

Executando VACUUM ANALYZE na tabela 'movie'...
Estatísticas atualizadas com sucesso!
Relatório de Metadados do Índice


,tabela,indice,tamanho,altura_arvore,bloco_raiz,total_blocos,total_linhas
0,movie,movie_key,56 kB,1,3,7,1844.0
1,movie,movie_title,96 kB,1,3,12,1844.0
2,movie,movie_votes,88 kB,1,3,11,1844.0



Sucesso! O arquivo 'relatorio_tarefa_11.txt' foi salvo na pasta do notebook.


NameError: name 'cur' is not defined

## Tarefa 12

In [16]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_teorica = """
RESPOSTA DA QUESTÃO 12.c:

O índice sobre o atributo VOTES não é utilizado em todas as consultas porque o otimizador
do PostgreSQL (Query Planner) avalia o custo total da operação.

1. Para consultas seletivas (poucas tuplas, <10), o custo de navegar na estrutura
   de árvore do índice (Index Scan) é baixo, e vale a pena para evitar ler a tabela toda.

2. Para consultas abrangentes (>80% das tuplas), o uso do índice geraria muitas operações
   de I/O aleatório (pular do índice para a tabela milhares de vezes). Nesse caso,
   é muito mais barato fazer um 'Sequential Scan' (ler a tabela inteira sequencialmente),
   aproveitando o cache do SO e a leitura em blocos contíguos.
"""

# Definição das Queries
queries = [
    {
        "titulo": "12.a) Consulta Pequena (<10 tuplas)",
        "sql": "SELECT title, votes FROM movie WHERE votes > 45000;" 
    },
    {
        "titulo": "12.b) Consulta Grande (>80% das tuplas)",
        "sql": "SELECT title, votes FROM movie WHERE votes > 10;"
    }
]

# execução e gravação do arquivo

nome_arquivo = "relatorio_tarefa_12.txt"

try:
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        
        f.write("BANCOS DE DADOS 1 - TRABALHO 3 - TAREFA 12\n")
        f.write("="*60 + "\n\n")

        # Loop para executar as duas consultas (a e b)
        for q in queries:
            print(f"Processando: {q['titulo']}...")
            
            # Escreve o título no arquivo
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY SQL: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa o EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            display(df)
            
            # Converte o DataFrame para string limpa (sem index, sem cabeçalho de coluna)
            # header=False remove o título 'QUERY PLAN' que é redundante
            plano_texto = df.to_string(index=False, header=False, justify='left')
            
            # Salva no arquivo
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # --- AQUI ENTRA A SUA RESPOSTA TEÓRICA ---
        f.write("ANÁLISE TEÓRICA (ITEM 12.c):\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_teorica)
        f.write("\n" + "="*60)

    print(f"\nSucesso! O arquivo '{nome_arquivo}' foi gerado com os planos e sua resposta.")

except Exception as e:
    print(f"Erro: {e}")

Processando: 12.a) Consulta Pequena (<10 tuplas)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes > 45000)
2,Planning Time: 0.121 ms
3,Execution Time: 0.011 ms


Processando: 12.b) Consulta Grande (>80% das tuplas)...


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=1844...
1,Filter: (votes > 10)
2,Planning Time: 0.022 ms
3,Execution Time: 0.178 ms



Sucesso! O arquivo 'relatorio_tarefa_12.txt' foi gerado com os planos e sua resposta.


## Tarefa 13

In [17]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """
RESPOSTA DA QUESTÃO 13 (Análise Comparativa):

a) Diferença entre os planos:
   SIM, houve uma diferença.
   - A Consulta A (MAX) utilizou um 'InitPlan' com 'Index Only Scan'. O banco calculou
     o valor máximo uma única vez (acessando o índice de forma invertida) e usou esse
     valor para filtrar a tabela.
   - A Consulta B (ALL) falhou em otimizar a lógica. O plano mostra um 'Seq Scan'
     (varredura sequencial) combinado com um 'SubPlan' que foi executado repetidamente
     (loops=1844), gerando um produto cartesiano lógico.

b) Qual das duas é mais eficiente?
   A Consulta A (MAX) é muito mais eficiente.
   - Custo estimado: ~35.37 (MAX) vs ~43620.99 (ALL).
   - Tempo real: 0.063ms (MAX) vs 0.632ms (ALL).

c) Explicação:
   Embora as consultas pareçam equivalentes, o operador 'ALL' possui uma semântica
   complexa de tratamento de valores NULL (se houver um NULL na subconsulta, o resultado
   pode ser indeterminado). O otimizador do PostgreSQL, neste cenário, não conseguiu
   garantir que a transformação de 'ALL' para 'MAX' seria segura ou vantajosa, apesar de
   não haver NULLs no DB (apenas ). Ele deicidiu optar por um plano genérico de verificação
   linha-a-linha (nested loop/materialize), o que elevou drasticamente o custo computacional.
"""

queries = [
    {
        "titulo" : "Consulta A - Usando MAX",
        "sql" : "SELECT title FROM movie WHERE votes >= (SELECT MAX(votes) FROM movie);"
    }, {
        "titulo" : "Consulta B - Usando ALL",
        "sql" : "SELECT title FROM movie WHERE votes >= ALL (SELECT votes FROM movie);"
    }
]

nome_arquivo = "relatorio_tarefa_13.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 13\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: Consulta A - Usando MAX...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes >= $1)
2,InitPlan 2 (returns $1)
3,-> Result (cost=0.32..0.33 rows=1 width=...
4,InitPlan 1 (returns $0)
5,-> Limit (cost=0.28..0.32 rows=1...
6,-> Index Only Scan Backward...
7,Index Cond: (votes IS ...
8,Heap Fetches: 0
9,Planning Time: 0.225 ms


Analisando: Consulta B - Usando ALL...


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..43620.99 rows=9...
1,Filter: (SubPlan 1)
2,Rows Removed by Filter: 1843
3,SubPlan 1
4,-> Materialize (cost=0.00..42.66 rows=18...
5,-> Seq Scan on movie movie_1 (cost...
6,Planning Time: 0.020 ms
7,Execution Time: 0.431 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_13.txt'.


## Tarefa 14

In [19]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """
RESPOSTA DA TAREFA 14 (Junção vs Seleção/Subconsulta):

a) Resultado do EXPLAIN ANALYZE:
   (Os planos detalhados estão apresentados acima).

b) Existe diferença entre os planos?
   SIM. Embora a lógica seja equivalente, a estrutura dos planos diferiu:
   
   1. Consulta A (Subconsulta): Utilizou um "InitPlan". O banco calculou os votos
      de 'Star Wars' ($0) isoladamente e antes da execução principal. Em seguida,
      usou esse valor escalar para realizar um 'Index Scan' simples na tabela movie.
      
   2. Consulta B (Junção): Utilizou um "Nested Loop" (Loop Aninhado). O banco
      primeiro localizou a tupla de 'Star Wars' (m2) e usou os dados dessa tupla
      para dirigir a busca na tabela m1.

c) Qual das duas é mais eficiente?
   Neste teste específico, a Consulta A (Subconsulta) foi ligeiramente mais eficiente.
   
   - Custo Total Estimado: 43.34 (A) vs 49.49 (B).
   - Tempo de Execução: 0.025ms (A) vs 0.034ms (B).
   
   Explicação: A estratégia de "InitPlan" (A) é extremamente otimizada para casos
   onde o filtro é um valor escalar constante. O "Nested Loop" (B), embora eficiente,
   possui um overhead ligeiramente maior para gerenciar a estrutura de junção,
   mesmo quando o loop externo retorna apenas uma linha. Contudo, para fins práticos
   em sistemas reais, ambas são consideradas de alta performance (sub-milissegundo).
"""

queries = [
    {
        "titulo": "14.a) Usando Subconsulta (Subquery)",
        "sql": "SELECT title FROM movie WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars')"
    },
    {
        "titulo": "14.b) Usando Junção (Self-Join)",
        "sql": "SELECT m1.title FROM movie m1, movie m2 WHERE m1.votes > m2.votes AND m2.title = 'Star Wars'"
    }
]

nome_arquivo = "relatorio_tarefa_14.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 14\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: 14.a) Usando Subconsulta (Subquery)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=8...
1,Index Cond: (votes > $0)
2,InitPlan 1 (returns $0)
3,-> Index Scan using movie_title on movie ...
4,Index Cond: ((title)::text = 'Star W...
5,Planning Time: 0.034 ms
6,Execution Time: 0.020 ms


Analisando: 14.b) Usando Junção (Self-Join)...


,QUERY PLAN
0,Nested Loop (cost=0.56..49.49 rows=615 width=...
1,-> Index Scan using movie_title on movie m2...
2,Index Cond: ((title)::text = 'Star War...
3,-> Index Scan using movie_votes on movie m1...
4,Index Cond: (votes > m2.votes)
5,Planning Time: 0.053 ms
6,Execution Time: 0.025 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_14.txt'.


## Tarefa 15

In [20]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)

resposta_teorica = """
RESPOSTA DA TAREFA 15 (Casamento de Strings e Índices):
RESPOSTA DA TAREFA 15 (Casamento de Strings e Índices):

a) Resultados do EXPLAIN:
   Todas as três consultas resultaram em 'Seq Scan' (Varredura Sequencial). O índice B-Tree
   sobre 'TITLE' não foi utilizado em nenhum dos casos.

b) Qual das três apresenta o menor custo?
   As consultas 15.1 (LIKE 'I%') e 15.3 (LIKE '%A') empataram com o menor custo estimado (38.05).
   A consulta 15.2 (substr) teve o MAIOR custo (42.66).
   
   Isso ocorre porque a função 'substr' exige processamento de CPU adicional para calcular
   a substring de cada linha antes de comparar, elevando o custo em relação à comparação
   simples de texto das outras duas. O tempo de execução real confirmou isso (0.168ms para substr
   vs 0.094ms para LIKE 'I%').

c) O índice sobre TITLE foi usado para todas elas? Justifique.
   NÃO. O índice não foi usado em nenhuma.

   1. LIKE 'I%': Teoricamente, índices B-Tree suportam buscas por prefixo. Porém, no PostgreSQL,
      se o Locale do banco não for 'C' (ex: pt_BR ou en_US), o índice padrão não suporta o operador LIKE
      devido a regras complexas de colação (ordenação). O otimizador, portanto, reverteu para Seq Scan.
      (Nota: Para funcionar, o índice precisaria ter sido criado com 'varchar_pattern_ops').

   2. substr(title, 1, 1): Índices B-Tree não suportam funções. Ao aplicar 'substr',
      o banco perde a referência da chave indexada e precisa calcular a função linha a linha.

   3. LIKE '%A': Índices B-Tree são ordenados pelo início da string. Como o curinga '%' está
      no início, a árvore não serve para a busca, forçando a leitura completa da tabela.
"""

queries = [
    {
        "titulo": "15.1) LIKE 'I%' (Tudo)",
        "sql": "SELECT title FROM movie WHERE title LIKE 'I%';"
    },
    {
        "titulo": "15.2) Substring (Função)",
        "sql": "SELECT title FROM movie WHERE substr(title, 1, 1) = 'I';"
    },
    {
        "titulo": "15.3) LIKE '%A' (Sufixo/Termina com A)",
        "sql": "SELECT title FROM movie WHERE title LIKE '%A';"
    }
]

# --- EXECUÇÃO E GERAÇÃO DO RELATÓRIO ---
nome_arquivo = "relatorio_tarefa_15.txt"

try:
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCOS DE DADOS 1 - TRABALHO 3 - TAREFA 15\n")
        f.write("CASAMENTO DE STRINGS E USO DE ÍNDICES\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Processando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            
            try:
                df = pd.read_sql(sql_explain, conn)
                display(df)
                plano_texto = df.to_string(index=False, header=False, justify='left')
                f.write(plano_texto)
            except Exception as e:
                f.write(f"Erro: {e}")
                
            f.write("\n\n" + "="*60 + "\n\n")
        
        f.write("ANÁLISE E RESPOSTAS (ITEM 15):\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_teorica)
        f.write("\n" + "="*60)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro geral: {e}")

finally:
    conn.close()

Processando: 15.1) LIKE 'I%' (Tudo)...


/tmp/ipykernel_6424/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=18 w...
1,Filter: ((title)::text ~~ 'I%'::text)
2,Rows Removed by Filter: 1819
3,Planning Time: 1.090 ms
4,Execution Time: 0.095 ms


Processando: 15.2) Substring (Função)...


/tmp/ipykernel_6424/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..42.66 rows=9 wi...
1,"Filter: (substr((title)::text, 1, 1) = 'I'::..."
2,Rows Removed by Filter: 1819
3,Planning Time: 0.061 ms
4,Execution Time: 0.272 ms


Processando: 15.3) LIKE '%A' (Sufixo/Termina com A)...


/tmp/ipykernel_6424/492076231.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_explain, conn)


,QUERY PLAN
0,Seq Scan on movie (cost=0.00..38.05 rows=18 w...
1,Filter: ((title)::text ~~ '%A'::text)
2,Rows Removed by Filter: 1814
3,Planning Time: 0.037 ms
4,Execution Time: 0.109 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_15.txt'.


## Tarefa 16

In [21]:
if conn.closed:
    conn = psycopg2.connect(dbname=db_name, **db_config)
    
resposta_tarefa_13 = """

a) Hipótese Uniforme:
O PostgreSQL não utilizou uma hipótese uniforme global. As estimativas apresentadas 
no EXPLAIN (rows=328 e rows=8) ficaram muito próximas dos valores reais (326 e 4), o que 
mostra que o otimizador consultou estatísticas reais armazenadas em pg_stats, como histogramas 
e valores mais comuns. No entanto, dentro de cada intervalo (bucket) do histograma, o 
PostgreSQL assume distribuição uniforme, e essa combinação de histogramas reais mais uniformidade interna 
explica por que as estimativas ficaram tão próximas dos valores observados.

b) Menor Seletividade:
A consulta "votes < 1000" possui a menor seletividade.
  Alta seletividade significa filtrar muito e retornar poucos dados (como "votes > 40000").
  Baixa seletividade significa filtrar pouco e retornar muitos dados (como "votes < 1000").

"""

queries = [
    {
        "titulo": "16.a) Votes < 1000 (muitos filmes)",
        "sql": "SELECT title FROM movie WHERE votes < 1000;"
    },
    {
        "titulo": "16.b) Votes > 4000 (poucos filmes)",
        "sql": "SELECT title FROM movie WHERE votes > 40000;"
    }
]

nome_arquivo = "relatorio_tarefa_16.txt"
try: 
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("BANCO DE DADOS 1 - TRABALHO 3 - TAREFA 14\n")
        f.write("="*60 + "\n\n")

        for q in queries:
            print(f"Analisando: {q['titulo']}...")
            
            f.write(f"CENÁRIO: {q['titulo']}\n")
            f.write(f"QUERY: {q['sql']}\n")
            f.write("-" * 30 + "\n")
            
            # Executa EXPLAIN ANALYZE
            sql_explain = f"EXPLAIN ANALYZE {q['sql']}"
            df = pd.read_sql(sql_explain, engine)
            
            # Mostra na tela para conferência rápida
            display(df)
            
            # Grava no arquivo
            plano_texto = df.to_string(index=False, header=False, justify='left')
            f.write(plano_texto)
            f.write("\n\n" + "="*60 + "\n\n")
        
        # Adiciona a resposta teórica no final
        f.write("ANÁLISE E RESPOSTAS:\n")
        f.write("-" * 30 + "\n")
        f.write(resposta_tarefa_13)

    print(f"\nSucesso! Relatório salvo em '{nome_arquivo}'.")

except Exception as e:
    print(f"Erro: {e}")

finally:
    conn.close()

Analisando: 16.a) Votes < 1000 (muitos filmes)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes < 1000)
2,Planning Time: 0.047 ms
3,Execution Time: 0.075 ms


Analisando: 16.b) Votes > 4000 (poucos filmes)...


,QUERY PLAN
0,Index Scan using movie_votes on movie (cost=0...
1,Index Cond: (votes > 40000)
2,Planning Time: 0.048 ms
3,Execution Time: 0.018 ms



Sucesso! Relatório salvo em 'relatorio_tarefa_16.txt'.


# Parte IV

## Tarefa 17